## **RCA and LCA separation from the original complete coronary tree segmentation**

The script identifies the RCA as the component with the rightmost center of mass and the LCA as the component with the leftmost center of mass.

This is an example for the case Normal 10.

### **Imports**

In [ ]:
import os
import numpy as np
import nibabel as nib
import scipy.ndimage as ndi
from skimage.measure import label, regionprops

### **Google Drive connection**

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Upload files**

In [ ]:
# Define input and output directories
test_input= '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/Skeletonization/GT/Normal_10.nii.gz'
test_output = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/RCA and LCA segmentations separation'

### **Functions**

In [ ]:
def separate_rca_lca(nifti_path, output_dir):
    """
    Separates the right coronary artery (RCA) and left coronary artery (LCA) from a NIfTI segmentation.
    RCA is identified as the rightmost component and LCA as the leftmost component based on center of mass.

    Parameters:
        nifti_path (str): Path to the input NIfTI file.
        output_dir (str): Path to the output directory where the separated segmentations will be saved.
    """
    # Load the segmentation NIfTI file
    img = nib.load(nifti_path)
    data = img.get_fdata()

    # Label connected component (A connectivity of 3 means that a voxel is considered connected to its 26 immediate neighbors (including diagonals in all three dimensions))
    labeled_data, num_labels = label(data, return_num=True, connectivity=3)
    print(f"Found {num_labels} connected components.")

    if num_labels < 2:
        print("Warning: Less than two connected components found. Check the input segmentation.")
        return

    # Compute center of mass for each component
    regions = regionprops(labeled_data)
    centers = [region.centroid for region in regions]

    # Identify RCA and LCA based on x-coordinate (left-right axis)
    x_coords = [center[0] for center in centers]
    rca_index = np.argmin(x_coords) + 1  # +1 because labels start from 1
    lca_index = np.argmax(x_coords) + 1  # +1 because labels start from 1

    print(f"RCA identified as component {rca_index}, LCA identified as component {lca_index}.")

    # Create binary masks for RCA and LCA
    rca_mask = (labeled_data == rca_index).astype(np.uint8)
    lca_mask = (labeled_data == lca_index).astype(np.uint8)

    # Save the separated masks as NIfTI files
    os.makedirs(output_dir, exist_ok=True)

    rca_nifti = nib.Nifti1Image(rca_mask, img.affine, img.header)
    lca_nifti = nib.Nifti1Image(lca_mask, img.affine, img.header)

    rca_path = os.path.join(output_dir, "RCA.nii.gz")
    lca_path = os.path.join(output_dir, "LCA.nii.gz")

    nib.save(rca_nifti, rca_path)
    nib.save(lca_nifti, lca_path)

    print(f"RCA saved at: {rca_path}")
    print(f"LCA saved at: {lca_path}")

### **Execution**

In [ ]:
# Example usage
separate_rca_lca(test_input, test_output)

Found 2 connected components.
RCA identified as component 1, LCA identified as component 2.
RCA saved at: /content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/RCA and LCA segmentations separation/RCA.nii.gz
LCA saved at: /content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/RCA and LCA segmentations separation/LCA.nii.gz
